# Tutorial pyAgrum

| | | |
|-|-|-|
|[ ![Creative Commons License](images/cc4.png)](http://creativecommons.org/licenses/by-nc/4.0/) |[ ![aGrUM](images/logoAgrum.png)](https://agrum.org) |[ ![interactive online version](images/atbinder.svg)](https://agrum.gitlab.io/extra/agrum_at_binder.html)


In [ ]:
from pylab import *
import matplotlib.pyplot as plt
import os

## Creating your first Bayesian network with pyAgrum

(This example is based on an OpenBayes [closed] website tutorial)

<p>A <b>Bayesian network</b> (BN) is composed of <b>random variables</b> (nodes) and their conditional dependencies (arcs) which, together, form a directed acyclic graph (DAG). A <b>conditional probability table</b> (CPT) is associated with each node. It contains the conditional probability distribution of the node given its parents in the DAG:

<img src="images/waterprinkler.png">

Such a BN allows to manipulate the joint probability $P(C,S,R,W)$&nbsp;&nbsp;&nbsp;using this decomposition :

$$P(C,S,R,W)=\prod_X P(X | Parents_X) = P(C) \cdot P(S | C) \cdot P(R | C) \cdot P(W | S,R)$$

Imagine you want to create your first Bayesian network, say for example the 'Water Sprinkler' network. This is an easy example. All the nodes are Boolean (only 2 possible values). You can proceed as follows.


### Import the pyAgrum package

In [ ]:
import pyAgrum as gum

### Create the network topology

#### Create the BN

The next line creates an empty BN network with a 'name' property.

In [ ]:
bn=gum.BayesNet('WaterSprinkler')
print(bn)

#### Create the variables

pyAgrum(aGrUM) provides 4 types of variables :

- LabelizedVariable
- RangeVariable
- IntegerVariable
- DiscretizedVariable

In this tutorial, we will use LabelizedVariable, which is a variable whose domain is a finite set of labels. The next line will create a variable named 'c', with 2 values and described as 'cloudy?', and it will add it to the BN. The value returned is the id of the node in the graphical structure (the DAG). pyAgrum actually distinguishes the random variable (here the labelizedVariable) from its node in the DAG: the latter is identified through a numeric id. Of course, pyAgrum provides functions to get the id of a node given the corresponding variable and conversely.

In [ ]:
c=bn.add(gum.LabelizedVariable('c','cloudy ?',2))
print(c)

You can go on adding nodes in the network this way. Let us use python to compact a little bit the code:

In [ ]:
s, r, w = [ bn.add(name, 2) for name in "srw" ] #bn.add(name, 2) === bn.add(gum.LabelizedVariable(name, name, 2))
print (s,r,w)
print (bn)

#### Create the arcs

Now we have to connect nodes, i.e., to add arcs linking the nodes. Remember that <tt>c</tt> and <tt>s</tt> are ids for nodes:

In [ ]:
bn.addArc(c,s)

Once again, python can help us :

In [ ]:
for link in [(c,r),(s,w),(r,w)]:
    bn.addArc(*link)
print(bn)

pyAgrum provides tools to display <tt>bn</tt> in more user-frendly fashions. <br/>Notably, <tt>pyAgrum.lib</tt> is a set of tools written in pyAgrum to help using aGrUM in python. <tt>pyAgrum.lib.notebook</tt> adds dedicated functions for iPython notebook.

In [ ]:
import pyAgrum.lib.notebook as gnb
bn

#### Shorcuts with fastBN

The functions `fast[model]` encode the structure of the graphical model and the type of the variables in a concise language somehow derived from the `dot` language for graphs (see the doc for the underlying method : [fastPrototype](https://pyagrum.readthedocs.io/en/latest/BNModel.html?highlight=fastPrototype#pyAgrum.BayesNet.fastPrototype)).

In [ ]:
bn=gum.fastBN("c->r->w<-s<-c")
bn

### Create the probability tables

Once the network topology is constructed, we must initialize the conditional probability tables (CPT) distributions.
Each CPT is considered as a Potential object in pyAgrum. There are several ways to fill such an object.<br/>

To get the CPT of a variable, use the cpt method of your BayesNet instance with the variable's id as parameter.<br/>

Now we are ready to fill in the parameters of each node in our network. There are several ways to add these parameters.<br/>

#### Low-level way

In [ ]:
bn.cpt(c).fillWith([0.4,0.6]) # remember : c=  0

Most of the methods using a node id will also work with name of the random variable.

In [ ]:
bn.cpt("c").fillWith([0.5,0.5])

#### Using the order of variables

In [ ]:
bn.cpt("s").names

In [ ]:
bn.cpt("s")[:]=[ [0.5,0.5],[0.9,0.1]]

Then $P(S | C=0)=[0.5,0.5]$ <br/>and $P(S | C=1)=[0.9,0.1]$.

In [ ]:
print(bn.cpt("s")[1])

The same process can be performed in several steps:

In [ ]:
bn.cpt("s")[0,:]=0.5 # equivalent to [0.5,0.5]
bn.cpt("s")[1,:]=[0.9,0.1]

In [ ]:
print(bn.cpt("w").names)
bn.cpt("w")

In [ ]:
bn.cpt("w")[0,0,:] = [1, 0] # r=0,s=0
bn.cpt("w")[0,1,:] = [0.1, 0.9] # r=0,s=1
bn.cpt("w")[1,0,:] = [0.1, 0.9] # r=1,s=0
bn.cpt("w")[1,1,:] = [0.01, 0.99] # r=1,s=1

#### Using a dictionnary

This is probably the most convenient way:

In [ ]:
bn.cpt("w")[{'r': 0, 's': 0}] = [1, 0]
bn.cpt("w")[{'r': 0, 's': 1}] = [0.1, 0.9]
bn.cpt("w")[{'r': 1, 's': 0}] = [0.1, 0.9]
bn.cpt("w")[{'r': 1, 's': 1}] = [0.01, 0.99]
bn.cpt("w")

The use of dictionaries is a feature borrowed from OpenBayes. It facilitates the use and avoid common errors that happen when introducing data into the wrong places.

In [ ]:
bn.cpt("r")[{'c':0}]=[0.8,0.2]
bn.cpt("r")[{'c':1}]=[0.2,0.8]

### Input/output

Now our BN is complete. It can be saved in different format :

In [ ]:
print(gum.availableBNExts())

We can save a BN using BIF format

In [ ]:
gum.saveBN(bn,"out/WaterSprinkler.bif")

In [ ]:
with open("out/WaterSprinkler.bif","r") as out:
    print(out.read())

In [ ]:
bn2=gum.loadBN("out/WaterSprinkler.bif")

We can also save and load it in other formats

In [ ]:
gum.saveBN(bn,"out/WaterSprinkler.net")
with open("out/WaterSprinkler.net","r") as out:
    print(out.read())
bn3=gum.loadBN("out/WaterSprinkler.net")

## Inference in Bayesian networks

We have to choose an inference engine to perform calculations for us. Many inference engines are currently available in pyAgrum:

- **Exact inference**, particularly :
    - `gum.LazyPropagation` : an exact inference method that transforms the Bayesian network into a hypergraph called a join tree or a junction tree. This tree is constructed in order to optimize inference computations.
    - others: `gum.VariableElimination`, `gum.ShaferShenoy`, ...
        
- **Samplig Inference** : approximate inference engine using sampling algorithms to generate a sequence of samples from the joint probability distribution (`gum.GibbSSampling`, etc.)
- **Loopy Belief Propagation** : approximate inference engine using inference algorithm exact for trees but not for DAG

In [ ]:
ie=gum.LazyPropagation(bn)

### Inference without evidence

In [ ]:
ie.makeInference()
print (ie.posterior("w"))

In [ ]:
from IPython.core.display import HTML
HTML(f"In our BN, $P(W)=${ie.posterior('w')[:]}")

With notebooks, it can be viewed as an HTML table

In [ ]:
ie.posterior("w")[:]

### Inference with evidence

Suppose now that you know that the sprinkler is on and that it is not cloudy, and you wonder what Is the probability of the grass being wet, i.e., you are interested in distribution $P(W|S=1,C=0)$. <br/>The new knowledge you have (sprinkler is on and it is not cloudy) is called evidence. Evidence is entered using a dictionary. When you know precisely the value taken by a random variable, the evidence is called a hard evidence. This is the case, for instance, when I know for sure that the sprinkler is on. In this case, the knowledge is entered in the dictionary as 'variable name':label

In [ ]:
ie.setEvidence({'s':0, 'c': 0})
ie.makeInference()
ie.posterior("w")

When you have incomplete knowledge about the value of a random variable, this is called a soft evidence. In this case, this evidence is entered as the belief you have over the possible values that the random variable can take, in other words, as <em>P(evidence|true value of the variable)</em>. Imagine for instance that you think that if the sprinkler is off, you have only 50% chances of knowing it, but if it is on, you are sure to know it. Then, your belief about the state of the sprinkler is [0.5, 1] and you should enter this knowledge as shown below. Of course, hard evidence are special cases of soft evidence in which the beliefs over all the values of the random variable but one are equal to 0.

In [ ]:
ie.setEvidence({'s': [0.5, 1], 'c': [1, 0]})
ie.makeInference()
ie.posterior("w") # using gnb's feature

the pyAgrum.lib.notebook utility proposes certain functions to graphically show distributions.

In [ ]:
gnb.showProba(ie.posterior("w"))

In [ ]:
gnb.showPosterior(bn,{'s':1,'c':0},'w')

### inference in the whole Bayes net

In [ ]:
gnb.showInference(bn,evs={})

#### inference with hard evidence

In [ ]:
gnb.showInference(bn,evs={'s':1,'c':0})

#### inference with soft and hard evidence

In [ ]:
gnb.showInference(bn,evs={'s':1,'c':[0.3,0.9]})

#### inference with partial targets

In [ ]:
gnb.showInference(bn,evs={'c':[0.3,0.9]},targets={'c','w'})

## Testing independence in Bayesian networks

One of the strength of the Bayesian networks is to form a model that allows to read qualitative knwoledge directly from the grap : the conditional independence. aGrUM/pyAgrum comes with a set of tools to query this qualitative knowledge.

In [ ]:
# fast create a BN (random paramaters are chosen for the CPTs)
bn=gum.fastBN("A->B<-C->D->E<-F<-A;C->G<-H<-I->J")
bn

## Conditional Independence

### Directly
First, one can directly test independence

In [ ]:
def testIndep(bn,x,y,knowing):
    res="" if bn.isIndependent(x,y,knowing) else " NOT"
    giv="." if len(knowing)==0 else f" given {knowing}."
    print(f"{x} and {y} are{res} independent{giv}")

testIndep(bn,"A","C",[])
testIndep(bn,"A","C",["E"])
print()
testIndep(bn,"E","C",[])
testIndep(bn,"E","C",["D"])
print()
testIndep(bn,"A","I",[])
testIndep(bn,"A","I",["E"])
testIndep(bn,"A","I",["G"])
testIndep(bn,"A","I",["E","G"])

### Markov Blanket

Second, one can investigate the Markov Blanket of a node. The Markov blanket of a node $X$ is the set of nodes $M\!B(X)$ such that $X$ is independent from the rest of the nodes given $M\!B(X)$.

In [ ]:
print(gum.MarkovBlanket(bn,"C").toDot())
gum.MarkovBlanket(bn,"C")

In [ ]:
gum.MarkovBlanket(bn,"J")

### Minimal conditioning set and evidence Impact using probabilistic inference

For a variable and a list of variables, one can find the sublist that effectively impacts the variable if the list of variables was observed.

In [ ]:
[bn.variable(i).name() for i in bn.minimalCondSet("B",["A","H","J"])]

In [ ]:
[bn.variable(i).name() for i in bn.minimalCondSet("B",["A","G","H","J"])]

This can be also viewed when using `gum.LazyPropagation.evidenceImpact(target,evidence)` which computes $P(target|evidence)$ but reduces as much as possible the set of needed evidence for the result :

In [ ]:
ie=gum.LazyPropagation(bn)
ie.evidenceImpact("B",["A","C","H","G"]) # H,G will be removed w.r.t the minimalCondSet above

In [ ]:
ie.evidenceImpact("B",["A","G","H","J"]) # "J" is not necessary to compute the impact of the evidence

### PS- the complete code to create the first image

In [ ]:
bn=gum.fastBN("Cloudy?->Sprinkler?->Wet Grass?<-Rain?<-Cloudy?")

bn.cpt("Cloudy?").fillWith([0.5,0.5])

bn.cpt("Sprinkler?")[:]=[[0.5,0.5],
                         [0.9,0.1]]

bn.cpt("Rain?")[{'Cloudy?':0}]=[0.8,0.2]
bn.cpt("Rain?")[{'Cloudy?':1}]=[0.2,0.8]

bn.cpt("Wet Grass?")[{'Rain?': 0, 'Sprinkler?': 0}] = [1, 0]
bn.cpt("Wet Grass?")[{'Rain?': 0, 'Sprinkler?': 1}] = [0.1, 0.9]
bn.cpt("Wet Grass?")[{'Rain?': 1, 'Sprinkler?': 0}] = [0.1, 0.9]
bn.cpt("Wet Grass?")[{'Rain?': 1, 'Sprinkler?': 1}] = [0.01, 0.99]

# the next line control the number of visible digits
gum.config['notebook','potential_visible_digits']=2
gnb.sideBySide(bn.cpt("Cloudy?"),captions=['$P(Cloudy)$'])
gnb.sideBySide(bn.cpt("Sprinkler?"),gnb.getBN(bn,size="3!"),bn.cpt("Rain?"),
              captions=['$P(Sprinkler|Cloudy)$',"",'$P(WetGrass|Sprinkler,Rain)$'])
gnb.sideBySide(bn.cpt("Wet Grass?"),captions=['$P(WetGrass|Sprinkler,Rain)$'])

### PS2- a second glimpse of `gum.config`

(for more, see the notebook : config for pyAgrum)

In [ ]:
bn=gum.fastBN("Cloudy?->Sprinkler?->Wet Grass?<-Rain?<-Cloudy?")

bn.cpt("Cloudy?").fillWith([0.5,0.5])

bn.cpt("Sprinkler?")[:]=[[0.5,0.5],
                         [0.9,0.1]]

bn.cpt("Rain?")[{'Cloudy?':0}]=[0.8,0.2]
bn.cpt("Rain?")[{'Cloudy?':1}]=[0.2,0.8]

bn.cpt("Wet Grass?")[{'Rain?': 0, 'Sprinkler?': 0}] = [1, 0]
bn.cpt("Wet Grass?")[{'Rain?': 0, 'Sprinkler?': 1}] = [0.1, 0.9]
bn.cpt("Wet Grass?")[{'Rain?': 1, 'Sprinkler?': 0}] = [0.1, 0.9]
bn.cpt("Wet Grass?")[{'Rain?': 1, 'Sprinkler?': 1}] = [0.01, 0.99]

# the next lines control the visualisation of proba as fraction
gum.config['notebook','potential_with_fraction']=True
gum.config['notebook', 'potential_fraction_with_latex']=True
gum.config['notebook', 'potential_fraction_limit']=100

gnb.sideBySide(bn.cpt("Cloudy?"),captions=['$P(Cloudy)$'])
gnb.sideBySide(bn.cpt("Sprinkler?"),gnb.getBN(bn,size="3!"),bn.cpt("Rain?"),
              captions=['$P(Sprinkler|Cloudy)$',"",'$P(WetGrass|Sprinkler,Rain)$'])
gnb.sideBySide(bn.cpt("Wet Grass?"),captions=['$P(WetGrass|Sprinkler,Rain)$'])